<a href="https://colab.research.google.com/github/eumarcosdeoliveira/Os-Amostradinhos/blob/Atividades/3_partitions_python_ailton_ribas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas pyarrow boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 3.1 MB/s eta 0:00:00


In [3]:
import boto3

# Configuração com as credenciais AWS
session = boto3.Session(
    aws_access_key_id='---',
    aws_secret_access_key='---' ,
    region_name='us-east-1'  # Escolha sua região AWS
)

# Inicializar o cliente S3
s3_client = session.client('s3')

In [4]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Exemplo de DataFrame de dados
data = {
    'nome': ['Alice', 'Bob', 'Charlie'],
    'idade': [24, 27, 22],
    'cidade': ['São Paulo', 'Rio de Janeiro', 'Curitiba']
}
df = pd.DataFrame(data)

# Converter DataFrame para tabela Parquet
table = pa.Table.from_pandas(df)


In [5]:
# Buffer em memória para armazenar o arquivo Parquet temporariamente
buffer = BytesIO()
pq.write_table(table, buffer)
buffer.seek(0)  # Mover o cursor para o início do buffer

# Nome do bucket e do arquivo no S3
bucket_name = 'ribashow666'
file_path = 'pasta_destino/dados.parquet'

# Cria o bucket se ele nao existir
try:
  s3_client.create_bucket(Bucket=bucket_name)
  print(f"Bucket '{bucket_name}' criado com sucesso.")
except Exception as e:
  print(f"Erro ao criar o bucket '{bucket_name}': {e}")
  pass

# Carregar o arquivo Parquet no S3
s3_client.upload_fileobj(buffer, bucket_name, file_path)
print(f"Arquivo salvo no S3 em: s3://{bucket_name}/{file_path}")


Bucket 'ribashow666' criado com sucesso.
Arquivo salvo no S3 em: s3://ribashow666/pasta_destino/dados.parquet


In [15]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
import os
import datetime

# Configuração do S3
partitioned_output_path = 'california_housing_partitioned/'  # Caminho dentro do bucket S3
bucket_name = 'ribashow666'  # Substitua pelo nome do seu bucket S3

# Carregar o dataset California Housing do arquivo CSV
# Certifique-se de que o arquivo CSV foi baixado do Kaggle ou é acessível localmente
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Exibir as primeiras linhas para entender o formato dos dados
df.head()

# As colunas do dataset incluem 'longitude', 'latitude', 'housing_median_age', 'total_rooms',
# 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value'

# Para particionamento, podemos usar por exemplo: 'median_income' (faixa de renda) ou 'housing_median_age'
# Aqui, vamos criar uma partição com base em 'housing_median_age' (faixa etária da casa)
# Vamos assumir que queremos particionar por faixa etária (categorizar em faixas)

df['housing_median_age'] = df['housing_median_age'].astype(int)

# Dividir em faixas de idade de casa (como exemplo para particionamento)
bins = [0, 10, 20, 30, 40, 50, 60, 100]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '60+']
df['age_group'] = pd.cut(df['housing_median_age'], bins=bins, labels=labels, right=False)

# Converter para uma tabela do PyArrow
table = pa.Table.from_pandas(df)

# Função para salvar a tabela particionada em S3
def save_partitioned_to_s3(table, bucket_name, output_path, partition_cols=['age_group']):
    # Salva os dados particionados em um diretório temporário local
    temp_output_path = "/tmp/california_housing_partitioned"
    pq.write_to_dataset(
        table,
        root_path=temp_output_path,
        partition_cols=partition_cols
    )

    # Criar cliente S3 with credentials
    session = boto3.Session(
        aws_access_key_id='---', # Replace with your AWS access key ID
        aws_secret_access_key='---', # Replace with your AWS secret access key
        region_name='us-east-1'  # Replace with your AWS region
    )
    s3_client = session.client('s3')


    # Para cada arquivo Parquet particionado, faça upload para o S3
    for root, dirs, files in os.walk(temp_output_path):
        for file in files:
            if file.endswith(".parquet"):
                local_file_path = os.path.join(root, file)
                # Calcula o caminho relativo dentro do diretório temporário
                relative_path = os.path.relpath(local_file_path, temp_output_path)
                s3_file_path = os.path.join(output_path, relative_path)

                # Carregar o arquivo Parquet para o S3
                with open(local_file_path, 'rb') as f:
                    s3_client.upload_fileobj(f, bucket_name, s3_file_path)
                print(f"Arquivo carregado no S3 em: s3://{bucket_name}/{s3_file_path}")

# Executar a função de salvamento particionado
save_partitioned_to_s3(table, bucket_name, partitioned_output_path)


Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=51-60/b63e91b78d16485eaeee97682c6f08b5-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=51-60/563c38735dbb4e72a80aa0e5eee89afd-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=51-60/5b41d822d66f4f60b053635149b8503f-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=41-50/b63e91b78d16485eaeee97682c6f08b5-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=41-50/563c38735dbb4e72a80aa0e5eee89afd-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=41-50/5b41d822d66f4f60b053635149b8503f-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_housing_partitioned/age_group=0-10/b63e91b78d16485eaeee97682c6f08b5-0.parquet
Arquivo carregado no S3 em: s3://ribashow666/california_